这是我用来处理数据的文件，其完成的任务就是生成一个最终版本的cb_table表格用于文件main进行读取和学习。整体的cb_table表格的形状为100000x74（100000是指ratings中的行数），有73列是属性，其中第0列是userid，第1列是movieid，第2列是我爬取的数据metascore，第3-22列是第一类标签，第23列到第72列是第二类标签tags，最后一列是一个0.5的倍数：评分。第二类标签只有50列的原因是：我利用glove预处理模型将每个电影的tags处理成一个长度为50的向量，然后将其作为电影的50个属性进行学习。
首先导入一些必要的库，其中torchtext库是用于文本处理、词嵌入的工作：

In [5]:
import pandas as pd
import numpy as np
import torch
import torchtext as tt

这些是处理数据必须用到的结构：将movieid转化成一个0-10000的数字，然后还需要设置一个从0-10000数字得到movieid的映射。这里的idToNum和NumToid就是这两个映射。cb_table作为全局变量，使得多个函数都可以对其进行修改。kind的含义之后会提及。

In [6]:
#global变量
idToNum = {}#从id变为一个有序的序列
NumToid = []#从具体的编号得到其id
cb_table = 0#cb_table为最终用于训练的属性、评分,最终写入cb_table.csv文件中
kind = 0

第一个函数就是connect_id_Num，就是建立movieid和修正id之间的映射。然后这里涉及到了全局变量kind。kind的作用是将电影中的genres列提取出来，以免之后再次读取movies.csv文件。读取出来之后，可以让fill_cb_table函数直接处理。

In [7]:
def connect_id_Num():#创建idToNum和NumToid
    global kind
    movies = pd.read_csv("movies.csv")
    temp = np.array(movies["movieId"])
    for i in range(len(temp)):
        idToNum[temp[i]] = i
        NumToid.append(temp[i])
    kind = list(movies["genres"])#把全局变量kind存储下来，用于之后填充cb_table[:,3-22]

build_cb_table函数的作用是初步搭建cb_table的一个框架，并填入一些基本的信息。在该函数中，首先创建了cb_table，并且读取了ans_table文件（爬取内容）。后续的操作是将userid、修正movieid、metascore和ratings先填入cb_table，之后便只剩下3-72行没有填入信息。

In [8]:
def build_cb_table():#初步搭建cb_table
    global cb_table
    ratings = np.array(pd.read_csv("ratings.csv"))
    cb_table = np.zeros((len(ratings),74),dtype = float)#返回结果

    ans_table = np.array(pd.read_csv("ans_table.csv"))#ans_table为爬取内容
    for i in range(len(cb_table)):
        cb_table[i,1] = idToNum[ratings[i,1]]#cb_table[:,1]为修正后的id
        cb_table[i,2] = ans_table[idToNum[ratings[i,1]],4]#cb_table[:,2]为爬取的metascore分数
    cb_table[:,0] = ratings[:,0]#cb_table[:,0]为用户id
    cb_table[:,73] = ratings[:,2]#cb_table[:,23]为评分


fill_cb_table函数的作用是利用先前创建的全局变量kind，将每一个标签都编上一个序号。标签种类一共有20种，每一个标签当作一个属性，其中有这个标签则属性为1，否则为0。下面的操作是获得每一部电影的标签种类，并且对cb_table文件进行修改。

In [9]:
def fill_cb_table():#把cb_table中空余的部分补充完整
    S = set()#用于存放电影类型（str）的集合
    for i in kind:#把所有电影类型拿出来，一共20部电影
        x = i.split("|")
        for j in x:
            S.add(j)
    NumOfTag = {}#每一个Tag对应cb_table中的第几列
    accum = 2#从2开始计数，3-22列为需要填充的部位
    for i in S:
        accum += 1
        NumOfTag[i] = accum
    for i in range(len(kind)):
        Id = NumToid[i]
        temp_index = np.where(cb_table[:,1] == Id)[0]
        x = kind[i].split("|")
        for j in x:
            cb_table[temp_index, NumOfTag[j]] = 1


这个函数的作用是：对导入cb_table的metascore进行处理。因为有的电影并没有metascore，这种时候我采取的填补缺失值策略是：覆盖上所有电影的平均值。此外，metascore的大小与其他属性的差距太大，因此这里我也对这一属性进行了标准化处理，将所有metascore归到均值为0、方差为1的正态分布中。

In [10]:
def deal_metascore():#处理爬取的metascore，进行填补缺失以及标准化处理
    def normalize(x,st,d):
        return (x-d)/st
    s1 = sum(cb_table[:,2] != 0)
    av1 = sum(cb_table[:,2]) / s1
    cb_table[cb_table[:,2] == 0, 2] = av1#处理缺失项
    st = np.std(cb_table[:,2])
    d = np.mean(cb_table[:,2])
    for i in range(len(cb_table)):
        cb_table[i,2] = normalize(cb_table[i,2], st, d)


这个函数是用于处理标签tags文件的函数，也是用到词嵌入方法的地方。
在这里，我首先定义了一个embedding类，类中只有一个embedding层，这个embedding层是直接从预训练的glove模型中创建的，因为该模型太大因此并没有传入作业文件中，而如果本目录下没有该模型，glove模型实例化的语句会自动下载该模型。用预处理glove词库，我们可以将一个单词转化为一个50维的词向量。
接下来是一个对tags预处理的函数first_deal。这个函数的功能是将一个字符串中的特殊字符（非数字，非字母）处理成空格，方便之后使用tokenizer进行分词。
接下来就是初始化分词器tokenizer，并创建我自己的词表vocab。将vocab词表传入embedding便可实例化一个Embedding网络。
接下来的操作是：对于一个tags（里面可能会有多个单词），对其中每一个单词都进行向量计算，最终将这些向量相加得到该tags的向量（虽然这会破坏单词的语义，但由于算力不够，只能进行此处理）。如果一部电影有多个tags，那么就对多个tags得到的向量进行平均值的求取而作为该电影的向量。如果没有被贴标签的电影，则向量直接置为0。
得到向量以后，便可以对cb_table中的23-72行进行填补。

In [11]:
def deal_tags():
    class Embedding(torch.nn.Module):#创建一个我的Embedding网络
        def __init__(self,vocab):
            super(Embedding,self).__init__()
            glove = tt.vocab.GloVe(name='6B',dim=50)#实例化glove模型
            self.embedding = torch.nn.Embedding.from_pretrained(
                glove.get_vecs_by_tokens(vocab.get_itos()),#将词表和glove模型整合
                freeze = True
            )
        def forward(self, a):
            return self.embedding(a)
    def first_deal(x):#字符串预处理，将非字母非数字的内容全部转化为空格
        x = x.lower()
        for i in range(len(x)):
            if not (('0'<=x[i] and '9'>=x[i]) or ('a'<=x[i] and 'z'>=x[i])):
                x = x[:i] + ' ' + x[i + 1:]
        return x
    tags = np.array(pd.read_csv("tags.csv"))
    tokenizer = tt.data.utils.get_tokenizer("basic_english")#初始化分词器
    word_list = [tokenizer(tags[i, 2]) for i in range(len(tags))]#分词器构建单词库
    vocab = tt.vocab.build_vocab_from_iterator(word_list)#创建词表
    E = Embedding(vocab)#初始化embedding网络
    for i in range(len(tags)):
        tags[i, 1] = idToNum[tags[i, 1]]#对于读入的tags表，先将movieid修正
        tags[i, 2] = first_deal(tags[i, 2])#处理字符串
    S = set(list(tags[:,1]))#获得所有被贴上标签的id集合
    for i in S:#i是一个电影id
        idx = np.where(tags[:,1] == i)[0]#找到tags里面id为i的索引
        length = len(idx)#length表示idx的长度，也即电影i被贴了几个标签
        temp = 0
        for j in idx:#j是索引中的一个，有tags[j,1]==i
            lt = word_list[j]#lt获取这个标签的分词列表（以空格分开）
            lt = [vocab[x] for x in lt]#将每个字转化为他的索引
            input_tensor = torch.tensor(lt)
            input_tensor = input_tensor.unsqueeze(0)
            output_tensor = E(input_tensor)
            output_tensor = output_tensor.squeeze(0)
            ttemp = 0
            for x in output_tensor:#一个标签lt可能由很多个单词组成，这里将他们
                ttemp += x
            temp += ttemp
        temp /= length
        idx = np.where(cb_table[:, 1] == i)[0]
        for j in idx:
            for k in range(50):
                cb_table[j, 23 + k] = temp[k]


查看在connect_id_Num函数执行后一些数据结构的情况：

In [12]:
connect_id_Num()
print(type(idToNum))
print(type(NumToid))

<class 'dict'>
<class 'list'>


In [13]:
build_cb_table()#初步搭建
print(cb_table[:5].shape)
cb_table[:5]

(5, 74)


array([[ 1.,  0., 96.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.],
       [ 1.,  2., 46.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.],
       [ 1.,  5., 76.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.

In [14]:
fill_cb_table()#填补3-22列
cb_table[:5]

array([[ 1.,  0., 96.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.],
       [ 1.,  2., 46.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.],
       [ 1.,  5., 76.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.

In [15]:
deal_metascore()#处理metascore
cb_table[:3]

array([[ 1.        ,  0.        ,  1.79998256,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0. 

In [16]:
deal_tags()#添加向量
cb_table[:2]

array([[ 1.00000000e+00,  0.00000000e+00,  1.79998256e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  8.71853352e-01,
        -1.04166996e+00, -6.92000007e-03,  4.73069996e-01,
         3.33009988e-01, -2.81479985e-01, -8.56400013e-01,
        -1.26901662e+00, -1.37054667e-01,  1.02336669e+00,
        -5.56016684e-01,  7.61323392e-01, -8.25756609e-01,
         4.16508317e-01,  7.99336672e-01, -1.85498998e-01,
         1.36203334e-01,  9.56720054e-01, -3.66590023e-01,
        -6.35653317e-01,  4.15416986e-01,  7.03486681e-01,
        -2.02686667e-01, -5.29666757e-03,  2.43153334e-01,
         1.69566676e-01, -1.03816664e+00, -5.08000469e-0

In [17]:
pd.DataFrame(cb_table).to_csv("cb_table2.csv")#存储cb_table